In [15]:
import os
import reservoirpy as rpy
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score
from reservoirpy.nodes import Reservoir
import logging
import gc
import pickle

# ログ設定
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# グローバル設定
rpy.verbosity(0)  # Verboseモードを無効
rpy.set_seed(42)  # 再現性のためのシード設定

In [16]:
# パラメータ設定
warm_up = 1000  # 初期値の捨てる数
max_delay = 2000  # 遅延数
fit_window = 1000  # W_outのfitするwindow
n_timesteps = warm_up + max_delay + fit_window  # 全体のtimesteps

rc_connectivity_list = np.array([0.2, 0.4, 0.6, 0.8, 1.0])
units_list = np.array([100, 500, 1000, 2000])
spectral_radii = np.array([0.1, 0.25, 0.5, 0.75, 1.0, 1.25])
batch_size = 100  # バッチサイズを適切に設定

# 結果保存先のディレクトリ
output_dir = "jupyter/src/memory_capacity/results/hyper_params_MC_modified"
os.makedirs(output_dir, exist_ok=True)  # ディレクトリがなければ作成

In [17]:
# データ生成
np.random.seed(42)
input_signal = np.random.randn(n_timesteps).reshape(-1, 1)  # ランダムな入力信号を生成
input_signal = (input_signal - np.mean(input_signal)) / np.std(input_signal)  # 標準化

In [18]:
# Reservoirを作成
def create_reservoir(spectral_radius: float, units: int, rc_connectivity: float) -> Reservoir:
    logging.info(f"Creating Reservoir with spectral radius={spectral_radius}, units={units} and rc_connectivity={rc_connectivity}")
    try:
        return Reservoir(
            units=units,
            input_scaling=1,
            rc_connectivity=rc_connectivity,
            lr=1,
            sr=spectral_radius,
            input_bias=False  # バイアス項を不要に設定
        )
    except Exception as e:
        logging.error(f"Failed to create ESN: {e}")
        return None

In [19]:
def fit_with_delay_and_get_r2(states_chunk: np.ndarray, target_chunk: np.ndarray, ridge: float = 1e-5) -> float:
    """
    指定された遅延に対して、リザバー状態から入力信号を予測し、R²スコアを計算する。

    Args:
        states_chunk (np.ndarray): 遅延後のリザバー状態行列。
        target_chunk (np.ndarray): 遅延前のターゲット信号。
        ridge (float): リッジ回帰の正則化パラメータ。

    Returns:
        float: R²スコア。
    """
    try:
        # リッジ回帰の計算
        reg = Ridge(alpha=ridge, fit_intercept=False)
        reg.fit(states_chunk, target_chunk)

        # 予測値の計算
        Y_pred = reg.predict(states_chunk)

        # R²スコアの計算
        r2 = r2_score(target_chunk, Y_pred)

        return r2
    except Exception as e:
        logging.error(f"Failed to fit model: {e}")
        return np.nan

def compute_r2_for_delay(states: np.ndarray, input_signal: np.ndarray, delay: int, fit_window: int) -> float:
    """
    遅延に対するR²スコアを計算。

    Args:
        states (np.ndarray): リザバー状態行列。
        input_signal (np.ndarray): 入力信号。
        delay (int): 遅延の大きさ。
        fit_window (int): フィッティングに使用するウィンドウサイズ。

    Returns:
        float: R²スコア。
    """
    try:
        if delay >= states.shape[0] - fit_window:
            return np.nan

        # 遅延後の状態とターゲット信号を取得
        states_chunk = states[delay:delay + fit_window, :]
        target_chunk = input_signal[:fit_window, :]

        # R²スコアを計算
        r2 = fit_with_delay_and_get_r2(states_chunk, target_chunk)
        return r2
    except Exception as e:
        logging.error(f"Error in computing R2 for delay {delay}: {e}")
        return np.nan

# 部分保存関数
def save_partial_results(result):
    try:
        spectral_radius = result["Spectral Radius"]
        units = result["Units"]
        rc_connectivity = result["RC Connectivity"]
        file_name = f"sr_{spectral_radius}_units_{units}_rc_{rc_connectivity}.pkl"
        path_name = os.path.join(output_dir, file_name)
        with open(path_name, "wb") as f:
            pickle.dump(result, f)
        logging.info(f"Partial result saved to {path_name}.")
    except Exception as e:
        logging.error(f"Failed to save partial result: {e}")

In [20]:
# メモリ容量を計算
def compute_memory_capacity(spectral_radius: float, units: int, rc_connectivity: float, input_signal: np.ndarray):
    logging.info(f"Calculating memory capacity for spectral radius={spectral_radius}, units={units} and rc_connectivity={rc_connectivity}")
    try:
        reservoir = create_reservoir(spectral_radius, units, rc_connectivity)
        if reservoir is None:
            raise Exception("Failed to create reservoir.")
        
        # リザバー状態を生成
        states = reservoir.run(input_signal).astype(np.float32)  # メモリ効率向上のため float32 にキャスト

        # 必要な部分のみを保持してメモリ使用量を削減
        states = states[warm_up:, :]
        input_signal_trimmed = input_signal[warm_up:, :]

        delays = np.arange(max_delay)
        r2_scores = []

        # 遅延ごとに並列処理
        r2_scores = Parallel(n_jobs=-1, backend="loky")(
            delayed(compute_r2_for_delay)(states, input_signal_trimmed, delay, fit_window) for delay in delays
        )

        memory_capacity = np.nansum(np.array(r2_scores, dtype=np.float32))
        result = {
            "Spectral Radius": spectral_radius,
            "Units": units,
            "RC Connectivity": rc_connectivity,
            "Memory Capacity": memory_capacity,
            "R2 Scores": r2_scores
        }
        save_partial_results(result)
        return result
    except Exception as e:
        logging.error(f"Failed to compute memory capacity for sr={spectral_radius}, units={units}, rc_connectivity={rc_connectivity}: {e}")
        return None


# 全てのパラメータの組み合わせについてメモリ容量を計算
def compute_all_memory_capacities(spectral_radii, units_list, rc_connectivity_list, input_signal):
    results = []
    params_list = [
        (sr, units, rc_connectivity)
        for sr in spectral_radii
        for units in units_list
        for rc_connectivity in rc_connectivity_list
    ]

    # パラメータの組み合わせごとに並列処理
    results = Parallel(n_jobs=-1, backend="loky")(
        delayed(compute_memory_capacity)(sr, units, rc_connectivity, input_signal)
        for sr, units, rc_connectivity in params_list
    )
    return results

In [21]:
# 実行
if __name__ == "__main__":
    all_results = compute_all_memory_capacities(spectral_radii, units_list, rc_connectivity_list, input_signal)
    print("All memory capacity results have been calculated and saved.")

All memory capacity results have been calculated and saved.
